In [6]:
import datetime
import rasterio
import utils


target_date = datetime.datetime(month=2, day=1, year=2022)
date_tolerance = 10 # plus and minus margin of target dates
max_cloud_percent = 30 # maximum amount of cloud permitted in a candidate scene

# the input raster, which defines our "area of interest"
AOI_raster = 'D:/Drone shapefile/classified_corrected/AcumulacionAaron2B_classified_corrected.tif'
ordername = AOI_raster.split("/")[-1].split(".")[0].split("_")[0]

# the directory where we store our whole satellite scenes
dl_directory = str('D:/sat_dls/')

In [7]:
start_date = target_date - datetime.timedelta(days=date_tolerance)
end_date = target_date + datetime.timedelta(days=date_tolerance)

if 'pl_API_key' not in globals():
    import getpass
    pl_API_key = getpass.getpass(prompt = 'Enter Planet API key: ')      # found at https://www.planet.com/account/#/user-settings

from shapely.geometry import Polygon, MultiPolygon

dataset = rasterio.open(AOI_raster)
raster_extent = utils.get_spatial_extent(dataset, latlon=True)

# limit on the number of scenes that will be ordered, here set to 2 for testing
#order_limit = 2

In [8]:
import planet_tools
pl_scenes = await planet_tools.pl_search_query(login=pl_API_key,
                            dataset='PSOrthoTile', # product type: see https://developers.planet.com/docs/apis/data/items-assets/
                            extent=raster_extent,
                            start_date=start_date,
                            end_date=end_date,
                            max_cloud_cover=max_cloud_percent,
                            name = ordername)

6 planet orders will be attempted


In [ ]:
dl_keys = utils.check_dls(dl_directory, planet=pl_scenes)
print("------")
for key, val in dl_keys.items():
    if len(val) > 0:
        print(val)
        if key == 'landsat':
            print('Downloading ' + key)
            ls_results = landsat_tools.ls_place_order(login=ls_login,
                                          scenes=val,
                                          dl_directory=dl_directory)
            output_directory = dl_directory + key
        if key == 'sentinel-2':
            print('Downloading ' + key)
            val = utils.s2_key_to_hash(val, s2_scenes)
            s2_results = sentinel_tools.s2_place_order(login=s2_login,
                                           scenes=val, 
                                           dl_directory=dl_directory)
            output_directory = dl_directory + key
        if key == 'planet':
            print('Downloading ' + key)
            val = utils.pl_key_to_hash(val, pl_scenes)
#            pl_results = [asyncio.create_task(planet_tools.pl_place_order(login=pl_API_key,
            pl_results = await planet_tools.pl_place_order(login=pl_API_key,
                                          scenes=val,
                                          dl_directory=dl_directory)
            output_directory = dl_directory + key

------
[['5329020_1933208_2022-01-27_2264', '5328549_1933208_2022-01-27_1049'], ['5371978_1933208_2022-02-08_2212'], ['5358149_1933208_2022-02-04_2427'], ['5351209_1933208_2022-02-02_2416'], ['5357953_1933208_2022-02-03_2426', '5354290_1933208_2022-02-03_105c'], ['5326082_1933208_2022-01-26_2251']]


00:00 - order  - state: creating

{'name': 'AcumulacionAaron2B', 'products': [{'item_ids': ['5329020_1933208_2022-01-27_2264', '5328549_1933208_2022-01-27_1049'], 'item_type': 'PSOrthoTile', 'product_bundle': 'analytic_sr_udm2'}]}
creating order


00:00 - order 2ab6f912-c687-464b-8a0f-9e0f29cb0aea - state: queued 

polling order


06:46 - order 2ab6f912-c687-464b-8a0f-9e0f29cb0aea - state: running

In [ ]:
# the directory where we want to store our clipped satellite scenes
import os
ordername = AOI_raster.split("/")[-1].split(".")[0].split("_")[0]
output_directory = str(dl_directory + ordername + "_clipped")

if not os.path.exists(output_directory):
    os.makedirs(output_directory)
top_directory_list = os.listdir(dl_directory)

In [ ]:
pl_scn_dirs = utils.pl_directory_scns(dl_directory + 'planet')

import planet_tools
raster_extent = utils.get_spatial_extent(dataset, latlon=False)
pg = MultiPolygon([Polygon(raster_extent)])
planet_tools.pl_crop(pl_scn_dirs, pg, output_directory)